# Basic Setting

In [0]:
from google.colab import files  #google colab上專用
import random
import numpy as np
import math                     #跑自然數字的函示exp()
import csv
import time
import heapq                    #來找一堆最大、小值的index
np.set_printoptions(suppress=True)

upload = files.upload()         #開啟上傳介面並載入(google colab上專用)
ExRate = np.loadtxt(open("SimExRatesFall19.csv", "rb"), delimiter=",", skiprows=1)  #將模擬匯率的CSV轉成matrix[1000*2]
putDM = np.array([[0.66,0.085855],[0.65,0.032191],[0.64,0.020795],
                  [0.63,0.017001],[0.62,0.013711],[0.61,0.010851],
                  [0.60,0.008388],[0.59,0.006291],[0.55,0.001401]])

putBP = np.array([[1.3,0.137213],[1.25,0.082645],[1.2,0.045060],
                  [1.15,0.028348],[1.1,0.016146],[1.05,0.007860],
                  [1.00,0.003277],[0.95,0.001134],[0.90,0.000245]])

n = 18                          #題目的要求，所以此數字不可以調。(numbers of decision variable)

Saving SimExRatesFall19.csv to SimExRatesFall19.csv


# Function A
### Random generate a solution with a set of decision variable and adjusted variable

In [0]:
def generate_a_solution():
  decision_var = []
  adjust_var = []
  for a in range(2):                        #有兩組決策變數，所以loop兩次
    temp_sum = 0
    temp_var = []
    for b in range(9):                      #並且一組為9個變數
      x = random.uniform(0,1)               #使用均勻分配
      temp_var.append(x)                    #將隨機生成的數丟到暫存list裡
      temp_sum = temp_sum + x               #紀錄9個變數的總和，以便於標準化
    for k in temp_var :
      decision_var.append(k/temp_sum)       #將標準化完的變數丟到真正的決策變數list裡
      adjust_var.append(first_adjust_var)   #將相對應數量的調整變數也丟到調整變數list裡
  
  output_var = decision_var+adjust_var      #合併決策變數list、調整變數list
  return(output_var)


# Function B
### Apply recombination and mutation to the parents to produce offspring

In [0]:
def mutation_step1(x,y):
  a = [x,y]
  b = random.sample(a,1)       #b為一個元素的一維陣列
  return b[0]            

In [0]:
def mutation_step2(x,y):
  a = (x+y)/2
  return a

In [0]:
def mutation_step3(x,k):                                                 #k為當輪交配之常態分佈產生的固定值
  a = np.random.standard_normal(1)                                       #a為一個元素的一維陣列
  b = math.exp(mutate_constant*k + mutate_constant*a[0])                 #a[0]這樣表示才會是一個數字
  c = x*b
  return c

In [0]:
def mutation_step4(array_decision,array_adjust):
  temp_sum = 0
  temp_var = []
  output_var = []
  for a in range(9):                                                                 #總共有9個決策變數，所以做9次
    decision_var = array_decision[a] + np.random.normal(0,array_adjust[a],1)[0]    
    while decision_var < 0 :                                                         ###(這個方法可換)當算出來的決策變數(尚未標準化前)小於零，則重新算一次，因為投資比例不能是負的
      decision_var = array_decision[a] + np.random.normal(0,array_adjust[a],1)[0]
    temp_var.append(decision_var)                           
    temp_sum = temp_sum + decision_var
  for b in temp_var :
    output_var.append(b/temp_sum)                                                    #將所有決策變數標準化
  return output_var

In [0]:
def mutation(matrix) : 
  output_var = []
  for i in range(kid_size):                                                     #需要幾個子代就做幾次
    temp_kid_decision_dm =[]
    temp_kid_decision_bp =[]
    kid_adjust_dm = []
    kid_adjust_bp = []
    indexList = range(pop_size)                                                 #隨機抽出兩個當父母-1
    randomIndex = random.sample(indexList, 2)                                   #隨機抽出兩個當父母-2
    parent1 = matrix[randomIndex[0],]
    parent2 = matrix[randomIndex[1],]
    kons = np.random.standard_normal(1)[0]                                      #論文中，equation(34)的固定參數
    for a in range(n+n):    #array/list的index從0開始
      if a < 9 :                                                                #前9個是DM的決策變數
        decision_dm_1 = mutation_step1(parent1[a] , parent2[a])
        temp_kid_decision_dm.append(decision_dm_1)
      elif a <18 :                                                              #第10~18個，是BP的決策變數
        decision_bp_1 = mutation_step1(parent1[a] , parent2[a])
        temp_kid_decision_bp.append(decision_bp_1)
      elif a <27 :                                                              #第19~27個，是DM的調節變數
        adjust_dm_1 = mutation_step2(parent1[a] , parent2[a])
        adjust_dm_2 = mutation_step3(adjust_dm_1, kons)
        kid_adjust_dm.append(adjust_dm_2)
      else :                                                                    #最後9個是BP的調節變數
        adjust_bp_1 = mutation_step2(parent1[a] , parent2[a])
        adjust_bp_2 = mutation_step3(adjust_bp_1, kons)
        kid_adjust_bp.append(adjust_bp_2)

    kid_decision_dm = mutation_step4(temp_kid_decision_dm , kid_adjust_dm)      #執行equation(35)-DM
    kid_decision_bp = mutation_step4(temp_kid_decision_bp , kid_adjust_bp)      #執行equation(35)-BP

    output_var = output_var + kid_decision_dm + kid_decision_bp + kid_adjust_dm + kid_adjust_bp

  output_matrix = np.array(output_var)
  output_matrix = output_matrix.reshape([kid_size, n+n])                        #從list轉換成array/矩陣
  return output_matrix

# Function C
### Use Hedged Problem objective function to determine the best offspring to constitute the population for the next generation

In [0]:
def hedge_step2BP(array_matrix,er):
  a_set_revenue=[]
  for c in range(array_matrix.shape[0]):
    an_option_revenue = 500*array_matrix[c]*(max(putBP[c,0] - 1.234*(1+er/100),0) - putBP[c,1])                #每個子代總共有9組解，對應到9個賣權選項的比例，所以unhedged revenue就需要針對每個賣權算一次，總計9次
    a_set_revenue.append(an_option_revenue)
  return sum(a_set_revenue)

In [0]:
def hedge_step2DM(array_matrix,er):
  a_set_revenue=[]
  for c in range(array_matrix.shape[0]):
    an_option_revenue = 500*array_matrix[c]*(max(putDM[c,0] - 0.6513*(1+er/100),0) - putDM[c,1])          #每個子代總共有9組解，對應到9個賣權選項的比例，所以unhedged revenue就需要針對每個賣權算一次，總計9次
    a_set_revenue.append(an_option_revenue)
  return sum(a_set_revenue)

In [0]:
def hedge_step1(array_matrix):
  temp_var=[]
  for b in range(ExRate.shape[0]):
    urDM = 645*0.6513*(1+ExRate[b,0]/100)
    urBP = 272*1.234*(1+ExRate[b,1]/100)
    hrDM_temp = hedge_step2DM(array_matrix[0:9],ExRate[b,0])
    hrBP_temp = hedge_step2BP(array_matrix[9:18],ExRate[b,1])
    each_sim_revenue = urDM + urBP + hrDM_temp + hrBP_temp
    temp_var.append(each_sim_revenue)                                           #總共有1000個匯率變化值，所以每個子代會有1000個profit
  output_var = sum(temp_var)/len(temp_var)                                      #全部加總起來算平均profit
  return output_var

In [0]:
def objective_function(kid_matrix):
  compare_var=[]
  output_next_gen_matrix = np.ones((pop_size,n+n))
  for a in range(kid_matrix.shape[0]):                                          
    mean_profit = hedge_step1(kid_matrix[a,])                                   #每個子代都算出一個平均profit，總共有kid_size個子代
    compare_var.append(mean_profit)

  next_gen_index = heapq.nlargest(pop_size, range(len(compare_var)), key=compare_var.__getitem__)       #找出前pop_size個最大平均profit的解之其原本的index的數值
  for k in range(len(next_gen_index)):
    output_next_gen_matrix[k,] = kid_matrix[next_gen_index[k],]                 #將優良的解變成下一代的母代matrix           

  output_best_set = kid_matrix[next_gen_index[0],0:18]
  output_max_profit = max(compare_var)
  return output_next_gen_matrix, output_best_set, output_max_profit

# Function D
### Random select a couple solutions of the last generation with a specific ratio and random generate the rest solutions to be the next-phase first parent solution

In [0]:
def turn_next_phase(better_matrix):
  next_phase_var = []
  number_of_pickup = int(pop_size*next_phase_ratio)                             #依照比率算出有幾個要變成下一階段的起始母代
  indexList = range(pop_size)                                                   #隨機挑選的步驟-1
  randomIndex = random.sample(indexList, number_of_pickup)                      #隨機挑選的步驟-2
  for a in range(len(randomIndex)):
    for b in better_matrix[randomIndex[a],]:                                    #這邊應該可以優化，但我先用我會的方式解決
      next_phase_var.append(b)
  for c in range(pop_size-number_of_pickup):
    next_phase_var = next_phase_var + generate_a_solution()
  output_matrix = np.array(next_phase_var)
  output_matrix = output_matrix.reshape([pop_size, n+n])
炳  return output_matrix

# Adjust the paremeters to find the maximum expected income

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
丙print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000567 0.00000038 0.00001466 0.00025731 0.00542219 0.00009172
 0.00009169 0.00000055 0.99411582 0.00000013 0.00001198 0.00000672
 0.00000238 0.0000067  0.00102983 0.07124918 0.92706188 0.0006312 ]
第10代的最大值 : 758.7573968569269
第10代結束
第20代的最佳解 : [0.00020226 0.00000471 0.00131284 0.01570437 0.000037   0.00016408
 0.00102976 0.00001677 0.98152821 0.00000029 0.00000003 0.00000553
 0.00005164 0.0000225  0.00074437 0.02435742 0.70429533 0.2705229 ]
第20代的最大值 : 758.8062320511729
第20代結束
第30代的最佳解 : [0.00000052 0.00000001 0.00000103 0.00013415 0.0000016  0.00000282
 0.00000218 0.00000317 0.99985452 0.         0.         0.
 0.00000061 0.00000019 0.         0.00003877 0.00331596 0.99664447]
第30代的最大值 : 758.981080704263
第30代結束
第1階段結束


0.9999999999999999
1.0
本次最佳解 : [0.00000052 0.00000001 0.00000103 0.00013415 0.0000016  0.00000282
 0.00000218 0.00000317 0.99985452 0.         0.         0.
 0.00000061 0.00000019 0.         0.00003877 0.00331596 0.99664447]
本次最大值 : 758.981080704263
總共跑

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00001033 0.00072143 0.00002236 0.0003011  0.00000116 0.00000084
 0.00002494 0.00044868 0.99846917 0.00000009 0.00000383 0.00036482
 0.00001186 0.00000199 0.00022281 0.00002344 0.9993404  0.00003076]
第10代的最大值 : 758.7937496141562
第10代結束
第20代的最佳解 : [0.00000019 0.00004097 0.00000093 0.00000039 0.00060588 0.00000039
 0.00006836 0.00042671 0.99885618 0.00000004 0.00000004 0.00001337
 0.00000042 0.00006471 0.00000137 0.00000002 0.85548158 0.14443846]
第20代的最大值 : 758.8248648228983
第20代結束
第30代的最佳解 : [0.         0.         0.         0.00000811 0.00000004 0.
 0.         0.00015618 0.99983568 0.00000228 0.         0.00000001
 0.         0.00000195 0.00000017 0.00000001 0.00003368 0.9999619 ]
第30代的最大值 : 758.9817371383814
第30代結束
第1階段結束


1.0
1.0
本次最佳解 : [0.         0.         0.         0.00000811 0.00000004 0.
 0.         0.00015618 0.99983568 0.00000228 0.         0.00000001
 0.         0.00000195 0.00000017 0.00000001 0.00003368 0.9999619 ]
本次最大值 : 758.9817371383814
總共跑了幾秒43.5696148

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00021021 0.00498776 0.01134089 0.00669406 0.02289076 0.00077091
 0.00486769 0.02509381 0.92314391 0.00000365 0.00002512 0.00000155
 0.00001137 0.00179818 0.0000022  0.00000517 0.00447779 0.99367498]
第10代的最大值 : 758.8687614082941
第10代結束
第20代的最佳解 : [0.         0.00000026 0.00000018 0.00008607 0.00000038 0.00001112
 0.00000657 0.00000005 0.99989537 0.         0.         0.00000009
 0.00000019 0.00000014 0.00000041 0.         0.00010307 0.99989609]
第20代的最大值 : 758.9817614085323
第20代結束
第30代的最佳解 : [0.         0.         0.         0.         0.         0.
 0.         0.00000004 0.99999996 0.00000004 0.         0.
 0.         0.00000001 0.00000005 0.00000002 0.0000064  0.99999347]
第30代的最大值 : 758.9819168794972
第30代結束
第1階段結束


1.0
1.0
本次最佳解 : [0.         0.         0.         0.         0.         0.
 0.         0.00000004 0.99999996 0.00000004 0.         0.
 0.         0.00000001 0.00000005 0.00000002 0.0000064  0.99999347]
本次最大值 : 758.9819168794972
總共跑了幾秒44.33864212036133


In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000087 0.00000068 0.00013083 0.00095723 0.00044561 0.00002697
 0.42580028 0.22430899 0.34832854 0.00014932 0.00025032 0.00000015
 0.001276   0.01134716 0.00008534 0.00010493 0.98677793 0.00000884]
第10代的最大值 : 758.2171288872657
第10代結束
第20代的最佳解 : [0.00001097 0.00000003 0.0000793  0.00183928 0.00007141 0.00002412
 0.00037606 0.00332596 0.99427285 0.00000063 0.00000036 0.00000018
 0.00001369 0.00033589 0.00009809 0.00416976 0.96955822 0.02582318]
第20代的最大值 : 758.79612166488
第20代結束
第30代的最佳解 : [0.00000001 0.         0.00000003 0.00000002 0.00000002 0.00000209
 0.         0.00000559 0.99999224 0.00000001 0.         0.0000138
 0.00009072 0.         0.00000123 0.00000001 0.00005676 0.99983748]
第30代的最大值 : 758.9815128517214
第30代結束
第1階段結束


1.0
1.0
本次最佳解 : [0.00000001 0.         0.00000003 0.00000002 0.00000002 0.00000209
 0.         0.00000559 0.99999224 0.00000001 0.         0.0000138
 0.00009072 0.         0.00000123 0.00000001 0.00005676 0.99983748]
本次最大值 : 758.9815128517214
總共跑了

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00758086 0.00000603 0.00012603 0.00092531 0.00017993 0.00083203
 0.10718711 0.00330788 0.87985482 0.00044026 0.00000259 0.00000006
 0.00092519 0.00019808 0.01169333 0.00000824 0.97447961 0.01225263]
第10代的最大值 : 758.4479026630776
第10代結束
第20代的最佳解 : [0.00005535 0.0000054  0.00007082 0.00238734 0.00032607 0.00011452
 0.00001148 0.02427941 0.97274961 0.00000291 0.         0.
 0.00000401 0.         0.0000004  0.0000001  0.14939439 0.85059818]
第20代的最大值 : 758.9325150615606
第20代結束
第30代的最佳解 : [0.00000093 0.00000075 0.00002813 0.00000567 0.         0.00010681
 0.00000023 0.00166168 0.99819579 0.00000438 0.00000055 0.00000002
 0.00000006 0.00000149 0.00000013 0.00000002 0.0013263  0.99866706]
第30代的最大值 : 758.980232368785
第30代結束
第1階段結束


0.9999999999999999
1.0
本次最佳解 : [0.00000093 0.00000075 0.00002813 0.00000567 0.         0.00010681
 0.00000023 0.00166168 0.99819579 0.00000438 0.00000055 0.00000002
 0.00000006 0.00000149 0.00000013 0.00000002 0.0013263  0.99866706]
本次最大值 : 758.98023236

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00010021 0.00162965 0.00045635 0.00001465 0.16017007 0.04963527
 0.17449302 0.00267089 0.61082989 0.00000353 0.00000694 0.00000009
 0.00097569 0.00000024 0.00272109 0.00083676 0.99545242 0.00000323]
第10代的最大值 : 758.3650554806017
第10代結束
第20代的最佳解 : [0.00000008 0.00000005 0.00000005 0.00000001 0.00000366 0.00000568
 0.00056816 0.00000082 0.99942149 0.         0.00000047 0.
 0.00000159 0.00000001 0.00008613 0.00037217 0.10334504 0.89619459]
第20代的最大值 : 758.9622109162218
第20代結束
第30代的最佳解 : [0.00000002 0.         0.00000014 0.00001198 0.00000009 0.00000291
 0.00000469 0.00000014 0.99998003 0.         0.         0.
 0.         0.         0.00000001 0.         0.00000465 0.99999534]
第30代的最大值 : 758.9818935769482
第30代結束
第1階段結束


1.0
1.0
本次最佳解 : [0.00000002 0.         0.00000014 0.00001198 0.00000009 0.00000291
 0.00000469 0.00000014 0.99998003 0.         0.         0.
 0.         0.         0.00000001 0.         0.00000465 0.99999534]
本次最大值 : 758.9818935769482
總共跑了幾秒45.13943696022034


In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000117 0.00001022 0.00001639 0.00000132 0.00171726 0.00000004
 0.00000908 0.00177165 0.99647287 0.00025721 0.00006761 0.00000401
 0.00056381 0.00000125 0.00004592 0.01472503 0.98246438 0.00187078]
第10代的最大值 : 758.7814615093367
第10代結束
第20代的最佳解 : [0.00003523 0.00012127 0.0000169  0.00000239 0.00014082 0.00000102
 0.00102925 0.00000147 0.99865165 0.00157429 0.00000001 0.0000069
 0.00000057 0.00006942 0.00000003 0.0016682  0.01894994 0.97773064]
第20代的最大值 : 758.9385691839499
第20代結束
第30代的最佳解 : [0.         0.         0.         0.         0.         0.
 0.         0.00000001 0.99999998 0.00000025 0.         0.
 0.00000001 0.00000008 0.00000009 0.00000477 0.00005283 0.99994197]
第30代的最大值 : 758.9819004064359
第30代結束
第1階段結束


1.0
1.0
本次最佳解 : [0.         0.         0.         0.         0.         0.
 0.         0.00000001 0.99999998 0.00000025 0.         0.
 0.00000001 0.00000008 0.00000009 0.00000477 0.00005283 0.99994197]
本次最大值 : 758.9819004064359
總共跑了幾秒44.593470335006714


In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00001017 0.00000148 0.00000088 0.00000119 0.00626197 0.0133163
 0.00738808 0.84768511 0.12533482 0.00000064 0.00019771 0.00133542
 0.00149249 0.00000981 0.00007375 0.00010148 0.00001109 0.99677762]
第10代的最大值 : 758.3580535350915
第10代結束
第20代的最佳解 : [0.00000224 0.00026431 0.00003522 0.00015461 0.00014841 0.00735805
 0.0003964  0.81551976 0.17612099 0.00000053 0.00003763 0.00181119
 0.00000408 0.00427958 0.0001359  0.00007852 0.00002348 0.99362908]
第20代的最大值 : 758.392507082882
第20代結束
第30代的最佳解 : [0.00000004 0.00000056 0.00000018 0.0000004  0.00000018 0.00000099
 0.00000005 0.00003939 0.9999582  0.         0.         0.
 0.         0.00000007 0.00000211 0.00000012 0.         0.9999977 ]
第30代的最大值 : 758.9818824071713
第30代結束
第1階段結束


1.0
1.0
本次最佳解 : [0.00000004 0.00000056 0.00000018 0.0000004  0.00000018 0.00000099
 0.00000005 0.00003939 0.9999582  0.         0.         0.
 0.         0.00000007 0.00000211 0.00000012 0.         0.9999977 ]
本次最大值 : 758.9818824071713
總共跑了幾秒44.944127321

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00001294 0.00001142 0.03612292 0.00066773 0.15635617 0.00093476
 0.00491838 0.05902576 0.74194992 0.00042085 0.00001941 0.00023585
 0.0182549  0.00030119 0.00011412 0.09752371 0.84304619 0.04008378]
第10代的最大值 : 758.3919044836123
第10代結束
第20代的最佳解 : [0.00000008 0.00001699 0.00000118 0.00015158 0.00010494 0.00000159
 0.04415662 0.00000205 0.95556497 0.00001167 0.00007862 0.00002263
 0.00006654 0.00070802 0.00060076 0.00042619 0.10668001 0.89140555]
第20代的最大值 : 758.9173008031348
第20代結束
第30代的最佳解 : [0.00000006 0.         0.00000173 0.         0.00000038 0.00000003
 0.00001032 0.00004434 0.99994313 0.00000055 0.00000041 0.00000216
 0.000015   0.00000019 0.00000275 0.00003174 0.00009707 0.99985013]
第30代的最大值 : 758.981750713922
第30代結束
第1階段結束


0.9999999999999999
1.0
本次最佳解 : [0.00000006 0.         0.00000173 0.         0.00000038 0.00000003
 0.00001032 0.00004434 0.99994313 0.00000055 0.00000041 0.00000216
 0.000015   0.00000019 0.00000275 0.00003174 0.00009707 0.99985013]
本次最大值 : 758.

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00005444 0.00000284 0.13349614 0.06814196 0.00042385 0.00632916
 0.00140231 0.00009019 0.79005911 0.00002841 0.00047207 0.00011227
 0.00018349 0.00869788 0.00240445 0.0000112  0.0008022  0.98728802]
第10代的最大值 : 758.6549889325092
第10代結束
第20代的最佳解 : [0.00014437 0.00154286 0.00017132 0.00871884 0.00389256 0.0109139
 0.0026686  0.04486688 0.92708067 0.0034815  0.00183324 0.05892766
 0.00125749 0.01148343 0.00308149 0.03020402 0.08307283 0.80665835]
第20代的最大值 : 758.4968757585834
第20代結束
第30代的最佳解 : [0.         0.00000002 0.         0.00000025 0.00002877 0.00000023
 0.00000157 0.00000353 0.99996563 0.         0.00000072 0.00002406
 0.00001211 0.00000015 0.00000138 0.00000004 0.00010856 0.99985297]
第30代的最大值 : 758.9817079924139
第30代結束
第1階段結束


0.9999999999999999
0.9999999999999999
本次最佳解 : [0.         0.00000002 0.         0.00000025 0.00002877 0.00000023
 0.00000157 0.00000353 0.99996563 0.         0.00000072 0.00002406
 0.00001211 0.00000015 0.00000138 0.00000004 0.00010856 0.9998529

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00001344 0.00000036 0.00018858 0.00001133 0.00006986 0.11737058
 0.00163384 0.00644076 0.87427124 0.0000572  0.00011855 0.00000259
 0.0223302  0.00049466 0.01868951 0.00120703 0.91860619 0.03849406]
第10代的最大值 : 758.5618028447358
第10代結束
第20代的最佳解 : [0.00000294 0.00000062 0.00039434 0.00030398 0.00466862 0.00056322
 0.00568078 0.00198587 0.98639964 0.00005212 0.00126748 0.00000644
 0.00033076 0.00000619 0.0001989  0.00011542 0.78944899 0.20857369]
第20代的最大值 : 758.8074530409655
第20代結束
第30代的最佳解 : [0.00000774 0.         0.00000229 0.00070737 0.00004639 0.00000071
 0.00008461 0.00006022 0.99909068 0.00043593 0.00000013 0.00000074
 0.01372425 0.00040855 0.00422686 0.00028521 0.00484676 0.97607157]
第30代的最大值 : 758.9112198505887
第30代結束
第1階段結束


1.0
1.0
本次最佳解 : [0.00000774 0.         0.00000229 0.00070737 0.00004639 0.00000071
 0.00008461 0.00006022 0.99909068 0.00043593 0.00000013 0.00000074
 0.01372425 0.00040855 0.00422686 0.00028521 0.00484676 0.97607157]
本次最大值 : 758.9112198505887


In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 30            #預設100代
max_phase = 1                   #預設5個階段
next_phase_ratio = 0.4          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00021293 0.00002511 0.00000147 0.01743216 0.000345   0.02714383
 0.00003293 0.00068506 0.9541215  0.00000647 0.00005499 0.00036268
 0.00000258 0.00000025 0.00002348 0.00022693 0.99932021 0.00000241]
第10代的最大值 : 758.7370662544374
第10代結束
第20代的最佳解 : [0.00000118 0.00000005 0.00000005 0.00000611 0.00000016 0.00003194
 0.00048791 0.00014625 0.99932634 0.00000025 0.00000013 0.00000152
 0.00000011 0.00000212 0.00004406 0.00111327 0.99881678 0.00002176]
第20代的最大值 : 758.7989355098227
第20代結束
第30代的最佳解 : [0.00000015 0.00000006 0.00000006 0.00000061 0.         0.00000057
 0.00000237 0.00000024 0.99999595 0.0000004  0.00000023 0.00000014
 0.         0.         0.00000042 0.00000263 0.99999599 0.00000018]
第30代的最大值 : 758.8001394008951
第30代結束
第1階段結束


1.0
0.9999999999999998
本次最佳解 : [0.00000015 0.00000006 0.00000006 0.00000061 0.         0.00000057
 0.00000237 0.00000024 0.99999595 0.0000004  0.00000023 0.00000014
 0.         0.         0.00000042 0.00000263 0.99999599 0.00000018]
本次最大值 : 758

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00002465 0.00000229 0.00040006 0.00006684 0.00029283 0.02712819
 0.01344272 0.00155076 0.95709166 0.00013584 0.00000029 0.00001858
 0.00063315 0.00012837 0.00571298 0.01434686 0.00087698 0.97814696]
第10代的最大值 : 758.9122909700443
第10代結束
第20代的最佳解 : [0.00006519 0.00005468 0.00013869 0.00010919 0.00000006 0.00000783
 0.00047037 0.0000657  0.99908829 0.00000291 0.00000008 0.00000002
 0.0000073  0.00000118 0.00002899 0.00400618 0.02228595 0.97366739]
第20代的最大值 : 758.9721030238109
第20代結束
第1階段結束


第10代的最佳解 : [0.00000031 0.00000527 0.00000003 0.0000003  0.00000001 0.00000171
 0.00000012 0.00000139 0.99999086 0.         0.         0.00000002
 0.00000008 0.         0.         0.00000649 0.0002245  0.99976891]
第10代的最大值 : 758.9818355861651
第10代結束
第20代的最佳解 : [0.00000003 0.         0.00000001 0.00000003 0.00000002 0.00000003
 0.         0.00000001 0.99999989 0.00000002 0.         0.
 0.         0.00000001 0.         0.00000117 0.00000002 0.99999877]
第20代的最大值 : 758.9819170069957
第20代結束
第2階

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.0000003  0.00000013 0.00000862 0.0259205  0.00001156 0.00003798
 0.00634505 0.05472287 0.91295299 0.0000043  0.00001209 0.00039078
 0.00002658 0.00001446 0.00328764 0.0000234  0.0000871  0.99615365]
第10代的最大值 : 758.8961432684308
第10代結束
第20代的最佳解 : [0.00000873 0.00000004 0.00000005 0.00038958 0.0000021  0.00005318
 0.00000602 0.01655883 0.98298146 0.         0.         0.00056791
 0.0006256  0.00011752 0.00000119 0.00121122 0.00785706 0.9896195 ]
第20代的最大值 : 758.9626090885338
第20代結束
第1階段結束


第10代的最佳解 : [0.         0.         0.         0.00000035 0.         0.
 0.00000081 0.00000001 0.99999883 0.         0.         0.00000001
 0.         0.         0.         0.00001389 0.00000047 0.99998563]
第10代的最大值 : 758.9819087413581
第10代結束
第20代的最佳解 : [0.         0.         0.         0.         0.         0.
 0.         0.00000005 0.99999995 0.         0.         0.
 0.         0.         0.         0.         0.         1.        ]
第20代的最大值 : 758.981919165015
第20代結束
第2階段結束


1.0
0.99999

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000262 0.00012076 0.06421664 0.00063643 0.00000527 0.00458302
 0.00000868 0.00008377 0.93034281 0.00022777 0.00010042 0.00116377
 0.00210337 0.00695902 0.01764746 0.00501487 0.95770138 0.00908194]
第10代的最大值 : 758.6401453582512
第10代結束
第20代的最佳解 : [0.00032397 0.00002868 0.00000473 0.0000271  0.00032814 0.00002187
 0.02443555 0.1504558  0.82437416 0.00000284 0.00000007 0.00000078
 0.00000028 0.00000002 0.00015713 0.00001347 0.00385103 0.99597439]
第20代的最大值 : 758.8454787979622
第20代結束
第1階段結束


第10代的最佳解 : [0.         0.00000001 0.00000615 0.00000009 0.00000992 0.00001558
 0.00000012 0.00005546 0.99991267 0.00000005 0.00029638 0.00000293
 0.00007006 0.00000038 0.00067471 0.00001293 0.21115214 0.78779042]
第10代的最大值 : 758.9387908339969
第10代結束
第20代的最佳解 : [0.         0.         0.0000017  0.00000001 0.00000005 0.00003955
 0.         0.00000114 0.99995755 0.         0.00000002 0.00000001
 0.00003866 0.00000003 0.00000245 0.00000017 0.00033731 0.99962136]
第20代的最大值 : 758.9816649295267
第2

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00004104 0.00000256 0.00002366 0.00000792 0.00126137 0.00061197
 0.00151366 0.00035669 0.99618115 0.00069485 0.00006039 0.00200864
 0.00001254 0.00018935 0.17131064 0.01227589 0.00059405 0.81285366]
第10代的最大值 : 758.6644645255088
第10代結束
第20代的最佳解 : [0.00000003 0.         0.00000623 0.00000001 0.         0.00000178
 0.0000495  0.00000001 0.99994244 0.         0.         0.00000001
 0.         0.         0.00004205 0.         0.0000483  0.99990964]
第20代的最大值 : 758.9817852444734
第20代結束
第1階段結束


第10代的最佳解 : [0.         0.         0.00000001 0.         0.         0.00000007
 0.00000012 0.         0.9999998  0.00000001 0.         0.00000027
 0.00000002 0.00000002 0.00000094 0.00000003 0.00000006 0.99999865]
第10代的最大值 : 758.9819159894424
第10代結束
第20代的最佳解 : [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
第20代的最大值 : 758.9819191988655
第20代結束
第2階段結束


1.0
1.0
本次最佳解 : [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
本次最大值 : 758.9819191988655
總共跑了幾秒59.15287137031555


In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00021255 0.00020817 0.00214009 0.00006649 0.00116958 0.00012663
 0.00037108 0.00010859 0.99559682 0.00269678 0.00246401 0.00263996
 0.00091978 0.01503973 0.02724347 0.00389397 0.88946303 0.05563928]
第10代的最大值 : 758.6156448709193
第10代結束
第20代的最佳解 : [0.00000276 0.00021047 0.00012238 0.00009813 0.00000078 0.00037235
 0.01315738 0.00043722 0.98559853 0.00000152 0.00007711 0.00000124
 0.00000001 0.00010938 0.00013038 0.00059705 0.01102847 0.98805485]
第20代的最大值 : 758.9640387689859
第20代結束
第1階段結束


第10代的最佳解 : [0.00000001 0.00000012 0.00000008 0.00000069 0.00000001 0.00000168
 0.         0.00000005 0.99999736 0.         0.00000679 0.
 0.00000011 0.00000073 0.         0.         0.00002887 0.9999635 ]
第10代的最大值 : 758.9818322473249
第10代結束
第20代的最佳解 : [0.         0.00000001 0.         0.         0.00000004 0.
 0.         0.         0.99999994 0.         0.         0.
 0.         0.00000001 0.         0.00000002 0.00000025 0.99999971]
第20代的最大值 : 758.981918912349
第20代結束
第2階段結束


1.0
1.0
本次最

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00678464 0.00826239 0.05635255 0.00337957 0.00064279 0.11614049
 0.00038235 0.10550001 0.70255521 0.00000022 0.00008394 0.00000393
 0.00201651 0.00045439 0.00005871 0.07826364 0.89211609 0.02700257]
第10代的最大值 : 758.2342851482475
第10代結束
第20代的最佳解 : [0.00061738 0.00002506 0.00023075 0.00000012 0.00417897 0.05679184
 0.00000104 0.00000735 0.93814748 0.00000303 0.00002772 0.00001991
 0.00005363 0.00067851 0.00003225 0.00036195 0.02702588 0.97179712]
第20代的最大值 : 758.8877861805993
第20代結束
第1階段結束


第10代的最佳解 : [0.         0.         0.0000002  0.00000002 0.00000027 0.00000419
 0.         0.00000107 0.99999425 0.00000013 0.00000129 0.00000002
 0.00000037 0.         0.00000036 0.00000009 0.0000032  0.99999454]
第10代的最大值 : 758.9818932387757
第10代結束
第20代的最佳解 : [0.         0.00000001 0.         0.         0.         0.
 0.         0.         0.99999998 0.         0.         0.
 0.         0.         0.         0.         0.         1.        ]
第20代的最大值 : 758.9819191405988
第20代結束
第2階段結束


1.

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000106 0.0002355  0.04361554 0.00204441 0.00298391 0.0002109
 0.9020352  0.00005034 0.04882313 0.00000749 0.00000307 0.00000119
 0.00001299 0.00001034 0.0000034  0.00003206 0.00182543 0.99810402]
第10代的最大值 : 758.0933905251178
第10代結束
第20代的最佳解 : [0.00000796 0.00007    0.00010026 0.00000284 0.00004644 0.00005628
 0.00730006 0.00056565 0.99185051 0.00000002 0.         0.00000003
 0.00000293 0.00000001 0.00000041 0.00000002 0.10012083 0.89987574]
第20代的最大值 : 758.9558574079199
第20代結束
第1階段結束


第10代的最佳解 : [0.         0.00000042 0.00000148 0.00000008 0.00006242 0.00000013
 0.00001098 0.00000052 0.99992397 0.         0.00000004 0.
 0.         0.         0.         0.         0.00000117 0.99999879]
第10代的最大值 : 758.9818255968963
第10代結束
第20代的最佳解 : [0.         0.         0.00000002 0.         0.00000003 0.
 0.00000006 0.         0.99999988 0.         0.         0.
 0.         0.         0.00000001 0.         0.00000003 0.99999996]
第20代的最大值 : 758.9819190533773
第20代結束
第2階段結束


1.0
0.99999

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000188 0.00435833 0.00007502 0.00164321 0.17675336 0.00103301
 0.54225784 0.06153521 0.21234213 0.00000833 0.00030987 0.00000307
 0.00210655 0.00135321 0.00125709 0.0276219  0.03212306 0.93521692]
第10代的最大值 : 758.1601485305129
第10代結束
第20代的最佳解 : [0.00000002 0.00000307 0.00000169 0.00000001 0.00000605 0.00000285
 0.00326071 0.02249583 0.97422976 0.         0.0000172  0.00002653
 0.00000937 0.00003228 0.00000209 0.00000036 0.01638495 0.98352722]
第20代的最大值 : 758.9601061932552
第20代結束
第1階段結束


第10代的最佳解 : [0.00000075 0.00000001 0.         0.00000003 0.0000001  0.
 0.00000124 0.00000311 0.99999475 0.00000002 0.00005473 0.
 0.00005254 0.00000001 0.0004674  0.00225451 0.00003012 0.99714067]
第10代的最大值 : 758.9788495122706
第10代結束
第20代的最佳解 : [0.00000001 0.00000006 0.         0.         0.00000007 0.00000001
 0.00000006 0.00000011 0.99999969 0.         0.         0.
 0.         0.         0.         0.         0.         0.99999999]
第20代的最大值 : 758.9819184803558
第20代結束
第2階段結束


1.0
1.0
本次

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000249 0.00001575 0.0001016  0.00002157 0.00050028 0.00056675
 0.00146173 0.99732983 0.         0.00000652 0.00000451 0.00025812
 0.00077086 0.00289116 0.00040346 0.00000671 0.00033572 0.99532295]
第10代的最大值 : 758.2823405046382
第10代結束
第20代的最佳解 : [0.00000001 0.00000104 0.00000008 0.00003713 0.00000013 0.00000129
 0.00000475 0.99994884 0.00000673 0.00000158 0.         0.0000002
 0.00002256 0.00000003 0.00000024 0.000011   0.         0.99996439]
第20代的最大值 : 758.2962336009373
第20代結束
第1階段結束


第10代的最佳解 : [0.         0.00000008 0.00000022 0.00000218 0.00000001 0.
 0.000006   0.99995611 0.0000354  0.         0.         0.
 0.         0.         0.         0.         0.         1.        ]
第10代的最大值 : 758.296408171041
第10代結束
第20代的最佳解 : [0.         0.00000058 0.00009433 0.00005886 0.00000002 0.00000079
 0.00000056 0.15766984 0.84217503 0.         0.         0.
 0.         0.         0.         0.         0.         1.        ]
第20代的最大值 : 758.8736143632965
第20代結束
第2階段結束


0.9999999999

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00080125 0.00049849 0.29309866 0.00258294 0.53322364 0.11125064
 0.00002639 0.00820082 0.05031716 0.00071305 0.00006113 0.0000068
 0.00008237 0.00099265 0.00047198 0.00896936 0.9886828  0.00001985]
第10代的最大值 : 757.5357779192736
第10代結束
第20代的最佳解 : [0.00000011 0.00000001 0.00035641 0.00000469 0.00125721 0.00024941
 0.00085549 0.00000854 0.99726814 0.         0.         0.
 0.00000042 0.00021671 0.00000003 0.00008959 0.99968599 0.00000726]
第20代的最大值 : 758.7963985129006
第20代結束
第1階段結束


第10代的最佳解 : [0.         0.         0.         0.00000002 0.00000011 0.00000002
 0.00000023 0.00000026 0.99999936 0.00000001 0.         0.
 0.00000001 0.00000066 0.         0.00012829 0.99987095 0.00000008]
第10代的最大值 : 758.8000989042331
第10代結束
第20代的最佳解 : [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
第20代的最大值 : 758.8001620772916
第20代結束
第2階段結束


1.0
1.0
本次最佳解 : [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
本次最大值 : 758.8001620772916
總共跑了幾秒58.422646284103394


In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.0003021  0.00000261 0.00969697 0.00749843 0.00000025 0.00120949
 0.00848178 0.12822858 0.84457979 0.00000011 0.00001884 0.00003444
 0.00007507 0.00000001 0.00000046 0.00025308 0.99956419 0.00005379]
第10代的最大值 : 758.6698252217516
第10代結束
第20代的最佳解 : [0.00000003 0.         0.         0.00002177 0.         0.00000001
 0.00008925 0.00000036 0.99988856 0.00000005 0.00000484 0.00000316
 0.         0.         0.00002941 0.00006323 0.02414138 0.97575792]
第20代的最大值 : 758.9772627102346
第20代結束
第1階段結束


第10代的最佳解 : [0.00000022 0.         0.         0.00000117 0.         0.00000001
 0.00000281 0.         0.99999579 0.         0.00000005 0.00000003
 0.00000001 0.         0.00000004 0.         0.00000288 0.99999699]
第10代的最大值 : 758.9819072561461
第10代結束
第20代的最佳解 : [0.         0.         0.         0.00000001 0.         0.
 0.         0.         0.99999999 0.         0.         0.
 0.         0.         0.         0.         0.00000002 0.99999998]
第20代的最大值 : 758.9819191597991
第20代結束
第2階段結束


1.

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.0000347  0.0008776  0.00001953 0.00844285 0.00004423 0.00000684
 0.00063436 0.00365154 0.98628835 0.00068316 0.00040619 0.00002123
 0.00000076 0.00241521 0.00021752 0.00076199 0.0580625  0.93743145]
第10代的最大值 : 758.9235898605348
第10代結束
第20代的最佳解 : [0.         0.00000017 0.         0.00006154 0.0000001  0.00000001
 0.00000002 0.00003824 0.99989991 0.00000869 0.         0.
 0.         0.00001432 0.00000474 0.00000091 0.00000397 0.99996736]
第20代的最大值 : 758.9815611548389
第20代結束
第1階段結束


第10代的最佳解 : [0.         0.         0.         0.         0.         0.
 0.         0.00000011 0.99999989 0.00000001 0.         0.
 0.         0.00000019 0.         0.00000003 0.0000012  0.99999856]
第10代的最大值 : 758.9819181630643
第10代結束
第20代的最佳解 : [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
第20代的最大值 : 758.9819191987436
第20代結束
第2階段結束


1.0
1.0
本次最佳解 : [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
本次最大值 : 758.9819191987436
總共跑了幾秒58.36536145210266


In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00005217 0.00018057 0.00079854 0.00041988 0.00000045 0.00071247
 0.44765419 0.55017497 0.00000677 0.00001173 0.00000001 0.00000088
 0.00001807 0.001147   0.0040738  0.00445046 0.00004893 0.99024912]
第10代的最大值 : 758.1805819419859
第10代結束
第20代的最佳解 : [0.00000062 0.00000149 0.00411153 0.00000033 0.00006817 0.00000189
 0.0000524  0.00000972 0.99575386 0.00000006 0.00000014 0.
 0.         0.00000542 0.00000001 0.00049065 0.00000002 0.99950369]
第20代的最大值 : 758.9756093264398
第20代結束
第1階段結束


第10代的最佳解 : [0.00000038 0.         0.000001   0.         0.00000137 0.00000071
 0.00001399 0.00000167 0.99998087 0.         0.00000001 0.
 0.         0.00000002 0.         0.00000581 0.00000006 0.9999941 ]
第10代的最大值 : 758.9818863986677
第10代結束
第20代的最佳解 : [0.         0.         0.         0.         0.00000012 0.00000001
 0.00000082 0.00000046 0.99999859 0.         0.         0.
 0.         0.         0.         0.00000007 0.00000002 0.99999991]
第20代的最大值 : 758.9819179183605
第20代結束
第2階段結束


0.99999999

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00000368 0.00014957 0.00140621 0.00160859 0.0158301  0.00240317
 0.00298747 0.00656013 0.96905109 0.00011148 0.01975073 0.00044379
 0.00053758 0.00876065 0.16547347 0.32700581 0.02873184 0.44918463]
第10代的最大值 : 758.2076327388556
第10代結束
第20代的最佳解 : [0.00000001 0.00001039 0.00000055 0.00000061 0.00002649 0.00051276
 0.01618452 0.00072224 0.98254243 0.         0.00000756 0.
 0.         0.00000008 0.00000001 0.00001158 0.00009848 0.99988229]
第20代的最大值 : 758.9659798432475
第20代結束
第1階段結束


第10代的最佳解 : [0.         0.         0.         0.         0.00000022 0.00000014
 0.00008135 0.00023486 0.99968341 0.00000127 0.00000654 0.00019638
 0.00000001 0.00000002 0.00008477 0.0003303  0.00442818 0.99495254]
第10代的最大值 : 758.9796417937752
第10代結束
第20代的最佳解 : [0.         0.         0.         0.         0.         0.
 0.00000005 0.00000004 0.99999991 0.         0.00000006 0.
 0.         0.         0.00000009 0.00000042 0.00004435 0.99995508]
第20代的最大值 : 758.9819100132629
第20代結束
第2階段結束


1.0
0.9999

In [0]:
#以下是可以調的參數
pop_size = n*1                  #預設7
kid_size = int(pop_size*1.7)           #預設7
max_generation = 20            #預設100代
max_phase = 2                   #預設5個階段
next_phase_ratio = 0.3          #預設0.1。目的為確認多少比例在換下一階段時保留
first_adjust_var = 3            #通常用3。目的為確認每一個階段的第一代之調整變數的數值
mutate_constant = 1.5             #論文上寫1。此數值為交配中的一個固定參數

start_time = time.time()
phase = 1
while phase <= max_phase:
  if phase ==1 :
    parent_var = []
    for a in range(pop_size):
      parent_var = parent_var + generate_a_solution()                                       #Function甲
    matrix = np.array(parent_var)
    matrix = matrix.reshape([pop_size, n+n])
  else :
    matrix = turn_next_phase(matrix)                                                        #Function丁
  for gen in range(max_generation):
    kid_matrix = mutation(matrix)                                                           #Function乙
    next_gen_parent_matrix, best_solution, max_profit = objective_function(kid_matrix)      #Function丙
    matrix = next_gen_parent_matrix
    if((gen+1)%10 == 0):
      print("第{}代的最佳解 : {}".format(gen+1, best_solution))
      print("第{}代的最大值 : {}".format(gen+1, max_profit))
      print("第{}代結束".format(gen+1))  
  print("第{}階段結束\n\n".format(phase))
  phase = phase + 1

exec_time =  time.time() - start_time
print(sum(next_gen_parent_matrix[3,0:9]))
print(sum(next_gen_parent_matrix[13,9:18]))
print("本次最佳解 : {}".format(best_solution))
print("本次最大值 : {}".format(max_profit))
print("總共跑了幾秒{}".format(exec_time))

第10代的最佳解 : [0.00002946 0.00005513 0.00437396 0.00642207 0.24096628 0.00065325
 0.00325506 0.03484464 0.70940015 0.0000128  0.00806617 0.00215523
 0.00794842 0.0023595  0.00155275 0.01043643 0.00303661 0.9644321 ]
第10代的最大值 : 758.4920062827322
第10代結束
第20代的最佳解 : [0.00001478 0.00010432 0.00001239 0.00009249 0.00001388 0.00008942
 0.00000147 0.00044123 0.99923004 0.00000166 0.00000085 0.00000023
 0.00000089 0.00001892 0.00007288 0.0000071  0.00013016 0.99976733]
第20代的最大值 : 758.9801612722874
第20代結束
第1階段結束


第10代的最佳解 : [0.00000001 0.00000006 0.00000364 0.00000073 0.00000024 0.00000257
 0.00000001 0.0000012  0.99999156 0.         0.00000005 0.00000001
 0.         0.00000003 0.00000006 0.00000043 0.00000142 0.999998  ]
第10代的最大值 : 758.9819072398543
第10代結束
第20代的最佳解 : [0.         0.         0.         0.         0.         0.
 0.         0.00000001 0.99999999 0.         0.         0.
 0.         0.         0.         0.00000002 0.00000002 0.99999995]
第20代的最大值 : 758.9819191335636
第20代結束
第2階段結束


1.